In [1]:
import pandas as pd

# Load the modified Excel file
df = pd.read_excel('（重要)手工清洗数据.xlsx', sheet_name = "Sheet1")

df.head(10)


,Department,Module Code,Activity Type,Activity Counts,Duration,Staff&activity count
0,SME,ACT2111,LEC,1,1.5h,Lu Yifei
1,SME,ACT3011,LEC,3,1.5h,Deng Tian(3)
2,SME,ACT3121,LEC,3,1.5h,Park Jong Han(3)
3,SME,ACT3131,LEC,2,1.5h,Shen Rui(2)
4,SME,ACT3141,LEC,2,1.5h,Zhou Yu#
5,SME,ACT3161,LEC,3,1.5h,Yoon Joon Sang(3)
6,SME,ACT3321,LEC,1,1.5h,Huang Kanyuan
7,SME,ACT4111,LEC,1,1.5h,Tsang Wilburn
8,SME,ACT4213,LEC,2,2h,Wang Xiaoqiao
9,SME,ACT4214,LEC,1,3h,Zhang Tianyu*


In [2]:
# Convert the Duration column to numeric values for comparison (assuming format like "1h", "2h")
df['Duration'] = df['Duration'].str.replace('h', '').astype(float)

# Sort data by all columns except 'Duration' and then 'Duration' in descending order to keep the largest
data_sorted = df.sort_values(by=['Department', 'Module Code', 'Activity Type', 'Activity Counts', 'Staff&activity count', 'Duration'], 
                               ascending=[True, True, True, True, True, False])

# Drop duplicates, keeping the entry with the maximum Duration
filtered_data = data_sorted.drop_duplicates(subset=['Department', 'Module Code', 'Activity Type', 'Activity Counts', 'Staff&activity count'], keep='first')

# Convert Duration back to string format if needed
filtered_data['Duration'] = filtered_data['Duration'].astype(str) + 'h'

# Replace Chinese parentheses with English ones in 'Staff&activity count'
filtered_data['Staff&activity count'] = filtered_data['Staff&activity count'].str.replace('（', '(', regex=False).str.replace('）', ')', regex=False)

# Display the first few rows of the filtered data
filtered_data.head()

,Department,Module Code,Activity Type,Activity Counts,Duration,Staff&activity count
80,MED,BIM2006,LEC,1,1.5h,Huang Zi Wei
75,MED,BIM3001,LEC,1,1.5h,"WANG Yongfei, SUN Hao (1)"
76,MED,BIM3009,LEC,1,1.5h,SUN Hao
78,MED,BIM3011,LEC,1,1.5h,Hirao Hajime
79,MED,BIM3019,LEC,1,1.5h,Chiang Ying-Chih


In [3]:
df = filtered_data
# Check if 'Staff&activity count' contains parentheses and modify if not
df['Staff&activity count'] = df.apply(
    lambda row: f"{row['Staff&activity count']} ({row['Activity Counts']})" 
    if '(' not in str(row['Staff&activity count']) else row['Staff&activity count'], axis=1
)

df.head(10)


,Department,Module Code,Activity Type,Activity Counts,Duration,Staff&activity count
80,MED,BIM2006,LEC,1,1.5h,Huang Zi Wei (1)
75,MED,BIM3001,LEC,1,1.5h,"WANG Yongfei, SUN Hao (1)"
76,MED,BIM3009,LEC,1,1.5h,SUN Hao (1)
78,MED,BIM3011,LEC,1,1.5h,Hirao Hajime (1)
79,MED,BIM3019,LEC,1,1.5h,Chiang Ying-Chih (1)
81,MED,BIO1001,LEC,1,1.5h,Yang Zhou (1)
82,MED,BIO2004,LEC,1,1.5h,Yongjuan Zhao (1)
83,MED,BIO2104,LAB,1,3.0h,"Zhou Yang, Yang Du, Yongjuan Zhao, Gang Chen, ..."
84,MED,BIO3201,LEC,1,1.5h,SUN Rui (1)
85,MED,BIO3204,LEC,1,1.5h,"DU Yang, LI Conglei (1)"


In [4]:
# Split 'Staff&activity count' by ';' and expand into separate rows
expanded_df = df.set_index([col for col in df.columns if col != 'Staff&activity count']).apply(
    lambda x: x['Staff&activity count'].split(';'), axis=1
).explode().reset_index()

# Rename the split column for clarity
expanded_df.rename(columns={0: 'Staff&activity count'}, inplace=True)

expanded_df.head(10)

,Department,Module Code,Activity Type,Activity Counts,Duration,Staff&activity count
0,MED,BIM2006,LEC,1,1.5h,Huang Zi Wei (1)
1,MED,BIM3001,LEC,1,1.5h,"WANG Yongfei, SUN Hao (1)"
2,MED,BIM3009,LEC,1,1.5h,SUN Hao (1)
3,MED,BIM3011,LEC,1,1.5h,Hirao Hajime (1)
4,MED,BIM3019,LEC,1,1.5h,Chiang Ying-Chih (1)
5,MED,BIO1001,LEC,1,1.5h,Yang Zhou (1)
6,MED,BIO2004,LEC,1,1.5h,Yongjuan Zhao (1)
7,MED,BIO2104,LAB,1,3.0h,"Zhou Yang, Yang Du, Yongjuan Zhao, Gang Chen, ..."
8,MED,BIO3201,LEC,1,1.5h,SUN Rui (1)
9,MED,BIO3204,LEC,1,1.5h,"DU Yang, LI Conglei (1)"


In [5]:
# # Rename 'Activity Counts' to 'Old Activity Counts'
# expanded_df.rename(columns={'Activity Counts': 'Old Activity Counts'}, inplace=True)

# Extract new Activity Counts from within parentheses in 'Staff&activity count' and convert to integer
expanded_df['new Activity Counts'] = expanded_df['Staff&activity count'].str.extract(r'\((\d+)\)').astype(int)

# Remove leading whitespace and extra spaces before parentheses in 'Staff&activity count'
expanded_df['Staff&activity count'] = expanded_df['Staff&activity count'].str.strip().str.replace(r'\s+\(', '(', regex=True)
expanded_df.tail(10)

# # Attempt to extract new Activity Counts from within parentheses in 'Staff&activity count'
# expanded_df['Activity Counts'] = pd.to_numeric(
#     expanded_df['Staff&activity count'].str.extract(r'\((\d+)\)')[0], errors='coerce'
# )

# # Filter out rows where 'Activity Counts' is NaN to identify problematic rows
# error_rows = expanded_df[expanded_df['Activity Counts'].isna()]

# # Display rows with errors
# error_rows


,Department,Module Code,Activity Type,Activity Counts,Duration,Staff&activity count,new Activity Counts
415,SSE,PHY2001,LEC,2,1.5h,Zhu Jian(1),1
416,SSE,PHY2210,LEC,1,1.0h,Ling Han(1),1
417,SSE,PHY2650,LEC,1,1.5h,Zhou Kai(1),1
418,SSE,PHY3007,LEC,1,1.5h,Zhang Zhaoyu(1),1
419,SSE,PHY3420,LEC,1,1.5h,"Wang Xing, staff_PHY3420(1)",1
420,SSE,PHY3610,LEC,1,1.5h,Cui Bingyu(1),1
421,SSE,PHY3710,LEC,1,1.5h,Randi Azmi(1),1
422,SSE,PHY4004,LEC,1,1.5h,Zhou Yan(1),1
423,SSE,PHY4221,LEC,1,1.5h,Shuai Zhigang(1),1
424,SSE,PHY4510,LEC,1,1.5h,Ji Yao(1),1


In [6]:

# Save to Excel with column width adjusted to header length
with pd.ExcelWriter("教授-初步清洗数据.xlsx", engine="openpyxl") as writer:
    expanded_df.to_excel(writer, index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # Set column width to match header length only
    for col in worksheet.iter_cols(min_row=1, max_row=1):
        max_length = len(str(col[0].value)) + 2  # Add some padding
        worksheet.column_dimensions[col[0].column_letter].width = max_length

In [7]:
split_data = expanded_df


In [8]:
#分裂成activity
# Initialize an empty list to store expanded rows
expanded_data_list = []

# Step 1: Duplicate each row based on 'Activity Counts'
for _, row in split_data.iterrows():
    count = int(row['new Activity Counts'])  # Get the number of times to duplicate the row
    # Duplicate the row 'count' times
    for _ in range(count):
        expanded_data_list.append(row.copy())

# Convert the list to a DataFrame
expanded_data = pd.DataFrame(expanded_data_list)

# Step 2: Apply sequential suffixes within each 'Module Code' group
final_data_list = []
for module_code, group in expanded_data.groupby('Module Code'):
    for idx, (_, row) in enumerate(group.iterrows(), start=1):
        count = int(row['Activity Counts'])
        if count > 1:
            # Copy the row and add a sequential suffix to 'Module Code'
            new_row = row.copy()
            new_row['Module Code'] = f"{module_code}_{str(idx).zfill(2)}"
            final_data_list.append(new_row)
        elif count == 1:
            final_data_list.append(row)

# Convert the list to a final DataFrame
final_data = pd.DataFrame(final_data_list)

# Display the first few rows of the modified data
final_data.head(10)


,Department,Module Code,Activity Type,Activity Counts,Duration,Staff&activity count,new Activity Counts
266,SME,ACT2111,LEC,1,1.5h,Lu Yifei(1),1
267,SME,ACT3011_01,LEC,3,1.5h,Deng Tian(3),3
267,SME,ACT3011_02,LEC,3,1.5h,Deng Tian(3),3
267,SME,ACT3011_03,LEC,3,1.5h,Deng Tian(3),3
268,SME,ACT3121_01,LEC,3,1.5h,Park Jong Han(3),3
268,SME,ACT3121_02,LEC,3,1.5h,Park Jong Han(3),3
268,SME,ACT3121_03,LEC,3,1.5h,Park Jong Han(3),3
269,SME,ACT3131_01,LEC,2,1.5h,Shen Rui(2),2
269,SME,ACT3131_02,LEC,2,1.5h,Shen Rui(2),2
270,SME,ACT3141_01,LEC,2,1.5h,Zhou Yu#(2),2


In [9]:
# Split 'Staff&activity count' by ',' and expand into separate rows
final_data = final_data.set_index([col for col in final_data.columns if col != 'Staff&activity count']).apply(
    lambda x: x['Staff&activity count'].split(','), axis=1
).explode().reset_index()

# Rename the split column for clarity
final_data.rename(columns={0: 'Staff&activity count'}, inplace=True)


# Remove leading and trailing whitespace from 'Staff&activity count'
final_data['Staff&activity count'] = final_data['Staff&activity count'].str.strip()

final_data.head(10)
final_data.to_excel("对照检验prof—activity.xlsx")

In [10]:
prof_activity = final_data
# 去除括号
# Remove parentheses and any numbers within them
prof_activity['Staff&activity count'] = prof_activity['Staff&activity count'].str.replace(r'\(\d+\)', '', regex=True).str.strip()
prof_activity = prof_activity[["Module Code","Staff&activity count"]]
# Rename the split column for clarity
prof_activity.rename(columns={'Staff&activity count': 'professor'}, inplace=True)
# Display the first few rows of the cleaned data
prof_activity.head()

# prof_activity.to_excel("prof-activity.xlsx", index= False)

# Save to Excel with column width adjusted to header length
with pd.ExcelWriter("prof-activity.xlsx", engine="openpyxl") as writer:
    prof_activity.to_excel(writer, index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # Set column width to match header length only
    for col in worksheet.iter_cols(min_row=1, max_row=1):
        max_length = len(str(col[0].value)) + 2  # Add some padding
        worksheet.column_dimensions[col[0].column_letter].width = max_length


C:\Users\ROG\AppData\Local\Temp\ipykernel_8964\4178985153.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prof_activity.rename(columns={'Staff&activity count': 'professor'}, inplace=True)
